In [ ]:
 !bw2 status

### Make an entity

This creates a keypair you can use to interact with BOSSWAVE resources.

We can add additional metadata to this entity, including
- Contact information (`-c`)
- Descriptive message (`-m`)
- Expiration (`-e`, defaults to 30 days)

This keypair is how we identify ourselves to BOSSWAVE resources. All BOSSWAVE operations (such as subscribing and publishing) are transparently verified by the routing infrastructure because they have a self-contained authenticating proof. This proof is generated by the BOSSWAVE "agent" running in this container, so you as a user will never have to interact directly with that construction.

In [ ]:
!bw2 mke -o myentity.ent

### Gain access to a namespace

We want to interact with resources on `$NAMESPACE`. We first verify if we can do so by running the following command:

In [ ]:
!echo $NAMESPACE
!bw2 bc -t myentity.ent -u $NAMESPACE/*

The `bc` command attempts to build a chain to see if there is a valid path in the DOT graph between the namespace entity and our own. It inspects the URI given by the `-u` argument to find the VK or alias of the entity responsible for administrating that namespace. Then, it traverses the graph of all DOTS granted by that entity until it finds a path of valid, non-expired DOTs that terminates at the VK for `myentity.ent`. This path is called a "DOT chain"

Now, we need to ask the owner of the namespace for permission to operate on their namespace. 

In [ ]:
!./getaccess <your VK here>

We've scripted this interaction for the purposes of the tutorial, but normally, gaining access to a namespace (or subset thereof) would be the outcome of a conversation with the namespace owner or some other individual with the ability to delegate permissions on that namespace.

The output of the above command will output something like 

```
DOT created
Hash:  Us19whkr4WXXrYojB5ytzKuKK_sqfmNh_2UWiTtj3mw=
Wrote dot to file:  .Us19whkr4WXXrYojB5ytzKuKK_sqfmNh_2UWiTtj3mw=.dot
Waiting for 1 object(s) to publish
Current BCIP set to 2 confirmation blocks or 20 block timeout
```

The `hash` represents the DOT object in the blockchain and is inspectable by the `bw2` command, as we observe below

Re-run this next line until "Registry" is marked **valid**

In [ ]:
!bw2 i <DOT HASH HERE>

When the registry field is valid, then we know the DOT object has been saved in the blockchain and is globally visible.

### Delegate access

We've already created a couple of entities for you --- `light.ent` and `switch.ent` --- that have permissions on $NAMESPACE

In [ ]:
!bw2 i light.ent
!bw2 bc -t light.ent -u $NAMESPACE/* -v

TODO: talk about the light stuff here

In [ ]:
!bw2 i switch.ent
!bw2 bc -t switch.ent -u $NAMESPACE/* -v

TODO: talk about the switch stuff here

**Here, create the light widget and have people publish to it using their own personal client**

Now, we can create a Python client with our entity and connect to our local agent.

We initialize the client with the entity file we want to use. We specify the entity file (and not our alias or VK) because the file contains the private signing key (SK) that we need for the agent to create proofs and sign our interactions with BOSSWAVE resources.

In [ ]:
from client import get_client
c = get_client(entity='myentity.ent')
import os
namespace = os.environ["NAMESPACE"]
c.publish(namespace+"/s.light/*/signal/state", "True")

### Writing our service

In BOSSWAVE, each process, device and individual has its own key. Because permissions are managed at the level of keys, this gives us fine-grained control over what can talk or listen to what.

To begin, we create a new entity for our service binding the switch to the light.

In [ ]:
!bw2 mke -o service.ent
!bw2 mkdot -f myentity.ent -t service.ent -u $NAMESPACE/s.light/light0/i.boolean/slot/state -x P
!bw2 mkdot -f myentity.ent -t service.ent -u $NAMESPACE/s.switch/switch0/i.boolean/signal/state -x C

In [ ]:
from client import get_client
c = get_client(entity='myservice.ent')
import os
namespace = os.environ["NAMESPACE"]

In [ ]:
def mycallback(msg):
    print msg.uri, msg.payload
c.subscribe(namespace+"/*", mycallback)

We give you a switch and a light. The switch publishes to its URL, and the light subscribes to its URL.

In [ ]:
from widgets import Widgets
w = Widgets()
w.display()

Now, you can wire the switch to the light using their URLs.

In [ ]:
import client

c = client.get_client()
def cb(msg):
    if msg.payload == "true":
        c.publish(w.light_url(), (64,0,0,1), "true")
    elif msg.payload == "false":
        c.publish(w.light_url(), (64,0,0,1), "false")

c.subscribe(w.switch_url(), cb)